# Introduction to Graph Datbases

The first part of this assignment is designed to give you hands-on experience with graph databases. You will start by setting up an in-memory graph database, for which the support code is already written. Once the database is running, you will execute queries of increasing complexity, exploring how relationships between nodes and edges are stored and retrieved. Through this process, you will gain practical insights into graph database concepts such as connectivity, traversal, and querying using graph-specific languages.

In [ ]:
%pip install kuzu

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))
from utils import setup_database, download_sample_data
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [3]:
# Download sample data for the Kuzudb example
data_dir = '../data'
download_sample_data(data_dir, urls=[
    "https://kuzudb.com/data/movie-lens/movies.csv",
    "https://kuzudb.com/data/movie-lens/users.csv",
    "https://kuzudb.com/data/movie-lens/ratings.csv",
    "https://kuzudb.com/data/movie-lens/tags.csv"
])

# Set up the Kuzudb database connection
connection = setup_database('../tmp', delete_existing=True)

# Create schema
connection.execute('CREATE NODE TABLE Movie (movieId INT64, year INT64, title STRING, genres STRING, PRIMARY KEY (movieId))')
connection.execute('CREATE NODE TABLE User (userId INT64, PRIMARY KEY (userId))')
connection.execute('CREATE REL TABLE Rating (FROM User TO Movie, rating DOUBLE, timestamp INT64)')
connection.execute('CREATE REL TABLE Tags (FROM User TO Movie, tag STRING, timestamp INT64)')

# Insert data
connection.execute(f'COPY Movie FROM "{data_dir}/movies.csv" (HEADER=TRUE)')
connection.execute(f'COPY User FROM "{data_dir}/users.csv" (HEADER=TRUE)')
connection.execute(f'COPY Rating FROM "{data_dir}/ratings.csv" (HEADER=TRUE)')
connection.execute(f'COPY Tags FROM "{data_dir}/tags.csv" (HEADER=TRUE)')


Saved https://kuzudb.com/data/movie-lens/movies.csv to ../data/movies.csv
Saved https://kuzudb.com/data/movie-lens/users.csv to ../data/users.csv
Saved https://kuzudb.com/data/movie-lens/ratings.csv to ../data/ratings.csv
Saved https://kuzudb.com/data/movie-lens/tags.csv to ../data/tags.csv
Sample data downloaded successfully
Loading graph database
Removing existing database at ../tmp


## Running Queries

Now that your graph database is set up, you can begin querying it. This section includes seven queries, each increasing in complexity.

In [4]:
# Query 1: Query all nodes with the label 'Movie'. Return those movie nodes. Limit your results to 25
result = connection.execute('MATCH (m:Movie) RETURN m LIMIT 25')

df = result.get_as_df()
df.head()

,m
0,"{'_id': {'offset': 0, 'table': 0}, '_label': 'Movie', 'movieId': 564, 'year': 1994, 'title': 'Chasers (1994)', 'genres': 'Comedy'}"
1,"{'_id': {'offset': 1, 'table': 0}, '_label': 'Movie', 'movieId': 567, 'year': 1993, 'title': 'Kika (1993)', 'genres': 'Comedy|Drama'}"
2,"{'_id': {'offset': 2, 'table': 0}, '_label': 'Movie', 'movieId': 568, 'year': 1993, 'title': 'Bhaji on the Beach (1993)', 'genres': 'Comedy|Drama'}"
3,"{'_id': {'offset': 3, 'table': 0}, '_label': 'Movie', 'movieId': 569, 'year': 1994, 'title': 'Little Big League (1994)', 'genres': 'Comedy|Drama'}"
4,"{'_id': {'offset': 4, 'table': 0}, '_label': 'Movie', 'movieId': 573, 'year': 1992, 'title': 'Ciao, Professore! (Io speriamo che me la cavo) (1992)', 'genres': 'Drama'}"


In [17]:
# Query 2: Query all nodes with the label 'Movie'. Get all connected nodes to the movie nodes. Limit your results to 50
# result = connection.execute('MATCH p = (e)-[]-(m:Movie) RETURN p LIMIT 50')
result = connection.execute('MATCH p = (m:Movie)<-[]-(e) RETURN p LIMIT 50')
# result = connection.execute('MATCH p = (m:Movie)-[]-(e) RETURN m, collect(e) LIMIT 50')

df = result.get_as_df()
df.head()

,p
0,"{'_nodes': [{'_id': {'offset': 0, 'table': 0}, '_label': 'Movie', 'movieId': 564, 'year': 1994, 'title': 'Chasers (1994)', 'genres': 'Comedy', 'userId': None}, {'_id': {'offset': 216, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 217}], '_rels': [{'_src': {'offset': 216, 'table': 1}, '_dst': {'offset': 0, 'table': 0}, '_label': 'Rating', '_id': {'offset': 30315, 'table': 2}, 'rating': 1.0, 'timestamp': 955945521, 'tag': None}]}"
1,"{'_nodes': [{'_id': {'offset': 0, 'table': 0}, '_label': 'Movie', 'movieId': 564, 'year': 1994, 'title': 'Chasers (1994)', 'genres': 'Comedy', 'userId': None}, {'_id': {'offset': 413, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 414}], '_rels': [{'_src': {'offset': 413, 'table': 1}, '_dst': {'offset': 0, 'table': 0}, '_label': 'Rating', '_id': {'offset': 61837, 'table': 2}, 'rating': 2.0, 'timestamp': 961438887, 'tag': None}]}"
2,"{'_nodes': [{'_id': {'offset': 1, 'table': 0}, '_label': 'Movie', 'movieId': 567, 'year': 1993, 'title': 'Kika (1993)', 'genres': 'Comedy|Drama', 'userId': None}, {'_id': {'offset': 131, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 132}], '_rels': [{'_src': {'offset': 131, 'table': 1}, '_dst': {'offset': 1, 'table': 0}, '_label': 'Rating', '_id': {'offset': 19969, 'table': 2}, 'rating': 2.0, 'timestamp': 1157992669, 'tag': None}]}"
3,"{'_nodes': [{'_id': {'offset': 1, 'table': 0}, '_label': 'Movie', 'movieId': 567, 'year': 1993, 'title': 'Kika (1993)', 'genres': 'Comedy|Drama', 'userId': None}, {'_id': {'offset': 201, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 202}], '_rels': [{'_src': {'offset': 201, 'table': 1}, '_dst': {'offset': 1, 'table': 0}, '_label': 'Rating', '_id': {'offset': 29087, 'table': 2}, 'rating': 3.0, 'timestamp': 974912341, 'tag': None}]}"
4,"{'_nodes': [{'_id': {'offset': 1, 'table': 0}, '_label': 'Movie', 'movieId': 567, 'year': 1993, 'title': 'Kika (1993)', 'genres': 'Comedy|Drama', 'userId': None}, {'_id': {'offset': 197, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 198}], '_rels': [{'_src': {'offset': 197, 'table': 1}, '_dst': {'offset': 1, 'table': 0}, '_label': 'Rating', '_id': {'offset': 28374, 'table': 2}, 'rating': 1.0, 'timestamp': 1034137937, 'tag': None}]}"


In [6]:
# Query 3: Count the total number of nodes in the database
# Hint: Use the `COUNT` function to count the number of nodes
result = connection.execute('MATCH (m) RETURN COUNT(m) AS total_nodes')

df = result.get_as_df()
print(df.head())

   total_nodes
0        10352


In [18]:
# Query 4: Query all nodes with the label 'User'. Count the degree for these nodes. Filter the nodes where the user rated more than 3 movies. Return the users and the degree
# Hint: First find all users and their ratings, then count the degree, and finally filter the results to only include users with more than 3 ratings
result = connection.execute('MATCH (u:User)-[r:Rating|:Tags]->(m:Movie) WITH u, COUNT(r) AS degree WHERE degree > 3 RETURN u, degree')

df = result.get_as_df()
df.head()

,u,degree
0,"{'_id': {'offset': 10, 'table': 1}, '_label': 'User', 'userId': 11}",64
1,"{'_id': {'offset': 19, 'table': 1}, '_label': 'User', 'userId': 20}",242
2,"{'_id': {'offset': 28, 'table': 1}, '_label': 'User', 'userId': 29}",81
3,"{'_id': {'offset': 29, 'table': 1}, '_label': 'User', 'userId': 30}",34
4,"{'_id': {'offset': 36, 'table': 1}, '_label': 'User', 'userId': 37}",21


In [19]:
# Query 5: Query all nodes with the label 'Movie'. Each node has a 'genre' attribute. Count the number of nodes per genre
# Hint: Use the `WITH` clause to group by genres and count the number of movies
result = connection.execute('MATCH (m:Movie) WITH m.genres AS genre, COUNT(m) AS count RETURN genre, count ORDER BY count DESC')

df = result.get_as_df()
df.head()

,genre,count
0,Drama,1058
1,Comedy,950
2,Comedy|Drama,435
3,Comedy|Romance,363
4,Drama|Romance,349


In [20]:
# Query 6: Query all nodes with the label 'Movie' and 'User', and the edge 'Rating' between movie and user. Each edge 'Rating' has a rating. Find the top 10 rated movies by average rating score
# Hint: Use the AVG clause to calculate an average. Use the `ORDER BY` clause to sort the movies by rating in descending order
result = connection.execute('MATCH (u:User)-[r:Rating]->(m:Movie) WITH m, AVG(r.rating) AS avg_rating RETURN m.title AS title, avg_rating ORDER BY avg_rating DESC LIMIT 10')

df = result.get_as_df()
df.head()

,title,avg_rating
0,Bloodsucking Bastards (2015),5.0
1,Ghost Graduation (2012),5.0
2,The Love Bug (1997),5.0
3,Saving Santa (2013),5.0
4,Obsession (1965),5.0


In [10]:
# Query 7: Query all nodes with the label 'Movie' and 'User', and the edge 'Rating' between movie and user. Find pairs of movies often rated by the same users
result = connection.execute('''
                            MATCH (u:User)-[r:Rating]->(m1:Movie), (u)-[r2:Rating]->(m2:Movie)
                            WHERE m1 < m2
                            RETURN m1.title AS Movie1, m2.title AS Movie2, COUNT(DISTINCT u) AS common_users
                            ORDER BY common_users DESC
                            LIMIT 10
                            ''')

df = result.get_as_df()
df.head()

,Movie1,Movie2,common_users
0,"Shawshank Redemption, The (1994)",Forrest Gump (1994),231
1,Pulp Fiction (1994),Forrest Gump (1994),230
2,Pulp Fiction (1994),"Shawshank Redemption, The (1994)",222
3,"Silence of the Lambs, The (1991)",Pulp Fiction (1994),207
4,"Silence of the Lambs, The (1991)",Forrest Gump (1994),199


In [14]:
result = connection.execute('MATCH p = (e)-[r]-(m:Movie) WHERE m.year >= 2000 RETURN e, r, m LIMIT 50')

df = result.get_as_df()
df

,e,r,m
0,"{'_id': {'offset': 0, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 1}","{'_src': {'offset': 0, 'table': 1}, '_dst': {'offset': 5105, 'table': 0}, '_label': 'Rating', '_id': {'offset': 571, 'table': 2}, 'rating': 5.0, 'timestamp': 964983536, 'tag': None}","{'_id': {'offset': 5105, 'table': 0}, '_label': 'Movie', 'movieId': 3273, 'year': 2000, 'title': 'Scream 3 (2000)', 'genres': 'Comedy|Horror|Mystery|Thriller'}"
1,"{'_id': {'offset': 0, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 1}","{'_src': {'offset': 0, 'table': 1}, '_dst': {'offset': 1815, 'table': 0}, '_label': 'Rating', '_id': {'offset': 581, 'table': 2}, 'rating': 5.0, 'timestamp': 964980668, 'tag': None}","{'_id': {'offset': 1815, 'table': 0}, '_label': 'Movie', 'movieId': 3578, 'year': 2000, 'title': 'Gladiator (2000)', 'genres': 'Action|Adventure|Drama'}"
2,"{'_id': {'offset': 0, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 1}","{'_src': {'offset': 0, 'table': 1}, '_dst': {'offset': 8884, 'table': 0}, '_label': 'Rating', '_id': {'offset': 582, 'table': 2}, 'rating': 4.0, 'timestamp': 964980683, 'tag': None}","{'_id': {'offset': 8884, 'table': 0}, '_label': 'Movie', 'movieId': 3617, 'year': 2000, 'title': 'Road Trip (2000)', 'genres': 'Comedy'}"
3,"{'_id': {'offset': 0, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 1}","{'_src': {'offset': 0, 'table': 1}, '_dst': {'offset': 8990, 'table': 0}, '_label': 'Rating', '_id': {'offset': 589, 'table': 2}, 'rating': 4.0, 'timestamp': 964980694, 'tag': None}","{'_id': {'offset': 8990, 'table': 0}, '_label': 'Movie', 'movieId': 3744, 'year': 2000, 'title': 'Shaft (2000)', 'genres': 'Action|Crime|Thriller'}"
4,"{'_id': {'offset': 0, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 1}","{'_src': {'offset': 0, 'table': 1}, '_dst': {'offset': 3787, 'table': 0}, '_label': 'Rating', '_id': {'offset': 590, 'table': 2}, 'rating': 5.0, 'timestamp': 964981855, 'tag': None}","{'_id': {'offset': 3787, 'table': 0}, '_label': 'Movie', 'movieId': 3793, 'year': 2000, 'title': 'X-Men (2000)', 'genres': 'Action|Adventure|Sci-Fi'}"
5,"{'_id': {'offset': 1, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 2}","{'_src': {'offset': 1, 'table': 1}, '_dst': {'offset': 1815, 'table': 0}, '_label': 'Rating', '_id': {'offset': 597, 'table': 2}, 'rating': 4.0, 'timestamp': 1445714885, 'tag': None}","{'_id': {'offset': 1815, 'table': 0}, '_label': 'Movie', 'movieId': 3578, 'year': 2000, 'title': 'Gladiator (2000)', 'genres': 'Action|Adventure|Drama'}"
6,"{'_id': {'offset': 1, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 2}","{'_src': {'offset': 1, 'table': 1}, '_dst': {'offset': 4015, 'table': 0}, '_label': 'Rating', '_id': {'offset': 598, 'table': 2}, 'rating': 4.0, 'timestamp': 1445714952, 'tag': None}","{'_id': {'offset': 4015, 'table': 0}, '_label': 'Movie', 'movieId': 6874, 'year': 2003, 'title': 'Kill Bill: Vol. 1 (2003)', 'genres': 'Action|Crime|Thriller'}"
7,"{'_id': {'offset': 1, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 2}","{'_src': {'offset': 1, 'table': 1}, '_dst': {'offset': 5327, 'table': 0}, '_label': 'Rating', '_id': {'offset': 599, 'table': 2}, 'rating': 3.5, 'timestamp': 1445714960, 'tag': None}","{'_id': {'offset': 5327, 'table': 0}, '_label': 'Movie', 'movieId': 8798, 'year': 2004, 'title': 'Collateral (2004)', 'genres': 'Action|Crime|Drama|Thriller'}"
8,"{'_id': {'offset': 1, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 2}","{'_src': {'offset': 1, 'table': 1}, '_dst': {'offset': 6630, 'table': 0}, '_label': 'Rating',